In [7]:
import torch
from torch import nn
import torchvision
import torchvision.transforms as T
from torchinfo import summary

import matplotlib.pyplot as plt
import seaborn as sns

from torchvision import datasets
from torch.utils.data import DataLoader, Dataset
from pathlib import Path

import transformer_architecture as TA
import utils 

# device = "cuda"
device= "mps"

/opt/homebrew/anaconda3/envs/headmind/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [8]:
%matplotlib inline

In [9]:
class ConvertToRGB(object):
    def __call__(self, image):
        if image.mode == 'RGB':
            return image
        elif image.mode == 'L':
            return image.convert('RGB')
        else:
            raise ValueError(f'Unsupported image mode: {image.mode}')





transform = T.Compose([
    ConvertToRGB(),
    T.Resize((224, 224)),
    T.ToTensor(),
    # Add other transforms as needed
])

dataset = torchvision.datasets.Caltech101(root=Path.cwd(), 
                                          target_type = 'category', 
                                          transform = transform, 
                                          target_transform = None, 
                                          download= True)

dataloader = DataLoader(dataset, batch_size=32)

classes = dataset.categories
num_classes = len(classes)

Files already downloaded and verified


In [10]:
X, y = next(iter(dataloader))

# Train a first ViT

In [13]:
vit = TA.ViT(num_classes=num_classes)
optimizer = torch.optim.Adam(params=vit.parameters(),
                             lr=3e-3, # Base LR from Table 3 for ViT-* ImageNet-1k
                             betas=(0.9, 0.999), # default values but also mentioned in ViT paper section 4.1 (Training & Fine-tuning)
                             weight_decay=0.3) # from the ViT paper section 4.1 (Training & Fine-tuning) and Table 3 for ViT-* ImageNet-1k

loss_fn = nn.CrossEntropyLoss()

In [14]:
results = utils.train(model=vit,
                       train_dataloader=dataloader,
                       test_dataloader=dataloader,
                       optimizer=optimizer,
                       loss_fn=loss_fn,
                       epochs=10,
                       device=device)

  0%|          | 0/10 [07:54<?, ?it/s]


KeyboardInterrupt: 